### helper function

In [8]:
# set environment
from pathlib import Path
SRC_PATH = Path('/home/ec2-user/SageMaker/david/tdm-sentiment/src/')
import sys
sys.path.append(str(SRC_PATH))
from config import *
from train_model import EconomicClassifier
from tqdm.notebook import tqdm
from joblib import Parallel, delayed
import random
import pandas as pd


parser = tdm_parser.TdmXmlParser()

def concatenate_text(text_list):
    """
    Concatenates a list of strings into a single string separated by spaces.
    
    Args:
        text_list (list): A list of strings.
        
    Returns:
        str: A single concatenated string.
    """
    if isinstance(text_list, list):
        return ' '.join(text_list)
    elif isinstance(text_list, str):
        return text_list  # Already a string
    else:
        return ""  # Return empty string for non-list, non-string entries
    

def dataset_into_df(dataset_path, xml_file_list, parser, choose_set):
    """
    Reads XML files from a specified directory, extracts relevant data, and converts it into a DataFrame.

    Args:
        dataset_path (str): Path to the directory containing the dataset of articles in XML format.
        xml_file_list (list): A list of XML file names containing the articles.
        parser (object): An object with methods to parse XML into soup and extract data into dictionaries.
        choose_set (set): A set of sections to include in the DataFrame.

    Returns:
        pd.DataFrame: A DataFrame containing concatenated data from the specified XML files.
    """
    content_list = []  # Initialize a list to store content dictionaries

    # Iterate over XML files in the provided list
    #for file_name in xml_file_list:
        #file_path = f'{dataset_path}/{file_name}'  # Construct the full file path
        #soup = parser.get_xml_soup(file_path)  # Parse the XML file into soup
        #content_dict = xml_to_df.xml_to_dict(file_path)  # Extract content of the file into a dictionary
        
        #content_list.append()
        # Append the dictionary to the list if it meets the section criteria
        #if content_dict and content_dict['section'] in choose_set:
            #content_list.append(content_dict)

    # Convert the list of dictionaries into a DataFrame
    df = xml_to_df.load_df_from_xml(xml_file_list)
    #df = pd.DataFrame(content_list)
    # Concatenates a list of strings into a single string
    #df['Text'] = df['Text'].apply(concatenate_text)
    return df


def read_file_names_in_chunks1(input_file, chunk_size):
    """
    Open a text file and read each row as a file name in chunks.
    
    Args:
        input_file (str): The path to the input text file.
        chunk_size (int): The number of lines (file names) to read per chunk.
        
    Yields:
        list: A list of file names read from the text file, chunk by chunk.
    """
    with open(f'{input_file}.txt', 'r') as f:
        chunk = []
        for i, line in enumerate(f, 1):
            file_name = line.strip()  # Remove any leading/trailing whitespace/newlines
            if file_name:  # Check if line is not empty
                chunk.append(file_name)
            
            if i % chunk_size == 0:
                yield chunk
                chunk = []  # Reset chunk list for the next batch

        # Yield any remaining lines in the last chunk
        if chunk:
            yield chunk


def save_file_names_to_txt(file_names, output_file):
    """
    Save a list of file names into a text file, appending to the existing content if it exists.
    
    Args:
        file_names (list): A list of file names to save.
        output_file (str): The path to the output text file.
    """
    with open(f'{output_file}.txt', 'a') as f:  
        for file_name in file_names:
            f.write(file_name + '\n')
    return None


positive_set = {'money', 'business', 'finance/business', 'business; part b; business desk', 'financial'}
negative_set = {'outlook', 'arts & entertainment', 'style', 'movies', 'arts', 'sports'}#', 
choose_set = positive_set | negative_set


dataset_name_list = ['USATodayDavid', 'ChicagoTribune', 'Newyork20042023', 'TheWashingtonPostDavid', 'LosAngelesTimesDavid']
project_path = Path('/home/ec2-user/SageMaker/david/tdm-sentiment/')

corpus_peth = Path('/home/ec2-user/SageMaker/data/')
    

### get data into a data set 

In [9]:

data_chunks = []
# Process each chunk of file names
for file_chunk in xml_to_df.read_file_names_in_chunks(FILE_NAMES_PATH / 'all_dataset_file_names', chunk_size=10**5):
    # Sample a maximum of 10,000 files from each chunk or the total number in the chunk if fewer
    file_chunk = random.sample(file_chunk, min(10**4, len(file_chunk)))
    print(file_chunk[:10])
    # Initialize a progress bar
    with tqdm(total=len(file_chunk)) as pbar:
        # Process the files in the current chunk
        df = dataset_into_df(corpus_peth, file_chunk, parser, choose_set)  
        data_chunks.append(df)  # Append the resulting DataFrame to the list

        # Update the progress bar after each chunk is processed
        pbar.update(len(file_chunk))

# Concatenate all DataFrame chunks into one DataFrame after the loop
data = pd.concat(data_chunks, ignore_index=True)

# Display the first row of the collected DataFrame
print(data.head(1))



['USATodayDavid/408972247.xml', 'USATodayDavid/2617961854.xml', 'USATodayDavid/835062469.xml', 'USATodayDavid/2883909966.xml', 'USATodayDavid/408902161.xml', 'USATodayDavid/408877860.xml', 'USATodayDavid/409001955.xml', 'USATodayDavid/409002893.xml', 'USATodayDavid/2172870890.xml', 'USATodayDavid/2570240369.xml']


  0%|          | 0/10000 [00:00<?, ?it/s]

OSError: [Errno 36] File name too long: "['USATodayDavid/408972247.xml', 'USATodayDavid/2617961854.xml', 'USATodayDavid/835062469.xml', 'USATodayDavid/2883909966.xml', 'USATodayDavid/408902161.xml', 'USATodayDavid/408877860.xml', 'USATodayDavid/409001955.xml', 'USATodayDavid/409002893.xml', 'USATodayDavid/2172870890.xml', 'USATodayDavid/2570240369.xml', 'USATodayDavid/408995890.xml', 'USATodayDavid/1963722250.xml', 'USATodayDavid/864880876.xml', 'USATodayDavid/2234303414.xml', 'USATodayDavid/2679540818.xml', 'USATodayDavid/193968399.xml', 'USATodayDavid/2449098378.xml', 'USATodayDavid/408906205.xml', 'USATodayDavid/2370263797.xml', 'USATodayDavid/408865804.xml', 'USATodayDavid/408902031.xml', 'USATodayDavid/1925426513.xml', 'USATodayDavid/409007779.xml', 'USATodayDavid/1810962633.xml', 'USATodayDavid/2873810744.xml', 'USATodayDavid/2213428207.xml', 'USATodayDavid/408984383.xml', 'USATodayDavid/409041316.xml', 'USATodayDavid/2243982715.xml', 'USATodayDavid/2312182578.xml', 'USATodayDavi

In [ ]:
df = xml_to_df.load_df_from_xml(xml_file_list)

### build model

In [ ]:
# initialize model
classifier = EconomicClassifier(initialize=True)
X_test, y_test = classifier.train_classifier(data)
classifier.save_model()

### evaluate model

In [1]:
evaluation = classifier.evaluate_model(X_test, y_test)
print(evaluation)

NameError: name 'classifier' is not defined

### use model

In [ ]:
# Initialize the classifier with the loaded model and vectorizer
classifier = EconomicClassifier()
article = "economy is expected to grow by 3% this year."
result = classifier.is_economic(article)
print(result)